In [ ]:
parallel_scenario = -1
# Parallelization options
# These can be overwritten by nbclick to run the notebook on the command line

# Designing and evaluating an online reinforcement learning agent for physical exercise recommendations in N-of-1 trials
This notebook is used to simulate the dataset for the evaluation of the RL agent.

## Imports & Constants

In [ ]:
#%load_ext autoreload
#%autoreload 2

In [ ]:
# Generic Imports
import pandas as pd
import pandas
import seaborn as sns
import matplotlib.pylab as plt
import numpy as np
import arviz as az
import arviz
import numpy
import hvplot
import holoviews
import random
import pymc as pm
import itertools
from tqdm.notebook import tqdm
from bokeh.io import show
from bokeh.layouts import column
import jsonpickle
import pymc
import datetime
print(datetime.datetime.now()) # Helps to see how long jobs took and when they were started

In [ ]:
# Simulation Library Imports
from adaptive_nof1 import SeriesOfSimulationsRunner, SeriesOfSimulationsData
from adaptive_nof1.models import Model, DiscretizedModel
from adaptive_nof1.policies import *
from adaptive_nof1.basic_types import *
from adaptive_nof1.metrics import *
from adaptive_nof1.inference import BayesianModel

In [ ]:
# Definition of constants:
baseline_duration = 1 * 7  # 1 week
phase_duration = 2 * 7  # 2 weeks
durations = [
    baseline_duration,
    phase_duration,
    phase_duration,
]
length = baseline_duration + 2 * phase_duration
n_patients = 100
noise_sigma = 1

posterior_update_interval = 1

In [ ]:
# Define intervention set
possible_actions = [
    # 30 min Jogging with different speeds
    {
        "intensity": 0.3,
        "duration": 0.5,
        "type": 0,
    },
    {
        "intensity": 0.5,
        "duration": 0.5,
        "type": 0,
    },
    {
        "intensity": 0.7,
        "duration": 0.5,
        "type": 0,
    },
    # HIT Training with different durations
    {
        "intensity": 1,
        "duration": 0.1,
        "type": 1,
    },
    {
        "intensity": 1,
        "duration": 0.2,
        "type": 1,
    },
    {
        "intensity": 1,
        "duration": 0.3,
        "type": 1,
    },
    # 45min Swimming
    {
        "intensity": 0.5,
        "duration": 0.75,
        "type": 2,
    },
    # 1h Yoga
    {
        "intensity": 0.1,
        "duration": 1,
        "type": 3,
    },
]
columns = ["intensity", "duration", "type"]
names_to_dimensions = {"type": 4}

## Data generation scenarios

We use different combinations of models to test the performance.
We always generate the context the same way, but the calculation of the pain reduction differs:
- I: Completely at random: Pain is drawn from N(0,1)
- II: Linear Model: Pain is calculated as a linear model with different coefficients drawn for each context and each action and patient.
- III: Activity-Indifferent Linear Model, Linear Model, but it is ignored which action is chosen 
- IV: Intensity-Indifferent Linear Model: Linear Model, but it is ignored which Intensity is chosen


In [ ]:
# This parent class defines how patient's contexts are generated
class ContextModel(Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.rng = np.random.default_rng(self.patient_id)

    def generate_context(self, history: History):
        last_intensities = [
            observation.treatment["intensity"]
            for observation in history.observations[-3:]
        ]
        last_durations = [
            observation.treatment["duration"]
            for observation in history.observations[-3:]
        ]
        return {
            # values derived from history
            "last_types": [
                observation.treatment["type"]
                for observation in history.observations[-3:]
            ],
            "last_intensities": last_intensities,
            "last_durations": last_durations,
            "mean_intensity": numpy.mean(last_intensities) if last_intensities else 0,
            "mean_duration": numpy.mean(last_durations) if last_durations else 0,
            # context variables
            "current_pain": self.rng.normal(0, 1),
        }

    def reset(self):
        self.rng = np.random.default_rng(self.patient_id)


# Scenario I:
class RandomScenario(ContextModel):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.rng = np.random.default_rng(self.patient_id)

    def __str__(self):
        return "Scenario I"

    def observe_outcome(self, action, context):
        return {"pain_reduction": self.rng.normal(0, noise_sigma), "compliant": True}


# Scenario II:
class LinearScenario(ContextModel):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        # Create patient by drawing random intercepts
        self.type_intercepts = self.rng.normal(0, 1, names_to_dimensions["type"])
        self.duration_coefficients = self.rng.normal(0, 1, 2)
        self.intensity_coefficients = self.rng.normal(0, 1, 2)
        self.pain_coefficients = self.rng.normal(0, 1, 2)

        self.noise_sigma = noise_sigma

        self.rng = np.random.default_rng(self.patient_id)

    def observe_outcome(self, action, context):
        action_type = action["type"]
        action_duration = action["duration"]
        action_intensity = action["intensity"]
        type_summand = self.type_intercepts[action_type]
        intensity_summand = (
            self.intensity_coefficients[0]
            + context["mean_intensity"] * self.intensity_coefficients[1]
        ) * action_intensity
        duration_summand = (
            self.duration_coefficients[0]
            + context["mean_duration"] * self.duration_coefficients[1]
        ) * action_duration
        pain_summand = (
            (
                self.pain_coefficients[0]
                + self.pain_coefficients[1] * context["current_pain"]
            )
            * action_intensity
            * action_duration
        )

        return {
            "pain_reduction": type_summand
            + intensity_summand
            + duration_summand
            + pain_summand
            + self.rng.normal(0, self.noise_sigma),
            "compliant": True
        }

    def __str__(self):
        return f"Scenario II"


# Scenario III:
class TypeIndifferentScenario(LinearScenario):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        # Use the same intercept (zero) for all types
        self.type_intercepts = numpy.repeat(0, len(self.type_intercepts))

    def __str__(self):
        return "Scenario III"

# Scenario IV:
class IntensityIndifferentScenario(LinearScenario):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        # Set coefficients that are related to intensity to 0
        self.intensity_coefficients = [0, 0]

    def __str__(self):
        return "Scenario IV"
    
# Scenario V:
class DurationIndifferentScenario(LinearScenario):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        # Set coefficients that are related to duration to 0
        self.duration_coefficients = [0, 0]

    def __str__(self):
        return "Scenario V"
    
# Scenario VI:
class IntensityAndDurationIndifferentScenario(LinearScenario):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        # Set all coefficients that are unrelated to type to 0
        self.intensity_coefficients = [0, 0]
        self.duration_coefficients = [0, 0]

    def __str__(self):
        return "Scenario VI"
    
# Scenario VII:
class NonAdherenceScenario(LinearScenario):
    def __str__(self):
        return "Scenario VII"
    
    def observe_outcome(self, action, context):
        linear_result = super().observe_outcome(action, context)
        if linear_result["pain_reduction"] > 0:
            # Drop 50% of "bad" results
            if self.rng.binomial(1, 0.5) > 0:
                linear_result["compliant"] = False
        return linear_result

# Define lambda functions "patient_id -> scenario" for runners
scenario_I = lambda patient_id: RandomScenario(patient_id=patient_id)
scenario_II = lambda patient_id: LinearScenario(patient_id=patient_id)
scenario_III = lambda patient_id: TypeIndifferentScenario(patient_id=patient_id)
scenario_IV = lambda patient_id: IntensityIndifferentScenario(patient_id=patient_id)
scenario_V = lambda patient_id: DurationIndifferentScenario(patient_id=patient_id)
scenario_VI = lambda patient_id: IntensityAndDurationIndifferentScenario(patient_id=patient_id)
scenario_VII = lambda patient_id: NonAdherenceScenario(patient_id=patient_id)

## Bayesian Model

In [ ]:
class PhysicalExerciseModel(BayesianModel):
    def __init__(
        self,
        dimension_for_type,
        possible_actions,
        **kwargs,
    ):
        self.possible_actions = possible_actions
        self.dimension_for_type = dimension_for_type
        self.action_name = "activity_index"
        self.coefficient_names = [
            "type",
            "intensity",
            "duration",
            "current_pain",
            "mean_duration",
            "mean_intensity",
        ]
        self.model = None
        super().__init__(**kwargs)

    def __str__(self):
        return "PhysicalExerciseModel"

    def data_to_treatment_indices(self, df):
        return pymc.intX((df[self.action_name]).to_numpy())

    def setup_model(self):
        empty_df = pandas.DataFrame(
            columns=list(
                numpy.unique(
                    [self.action_name] + self.coefficient_names + ["pain_reduction"]
                )
            ),
            dtype=float,
        )
        self.model = pymc.Model()
        with self.model:
            mu = 0
            type_intercept = pymc.Normal(
                "type_intercept",
                mu=0,
                sigma=1,
                dims="type_number",
                shape=self.dimension_for_type,
            )
            types = pymc.MutableData(
                "types", pymc.intX(empty_df["type"]), dims="obs_id"
            )
            intensities = pymc.MutableData(
                "intensities", pymc.floatX(empty_df["intensity"]), dims="obs_id"
            )
            mean_intensities = pymc.MutableData(
                "mean_intensities", pymc.floatX(empty_df["intensity"]), dims="obs_id"
            )
            durations = pymc.MutableData(
                "durations", pymc.floatX(empty_df["duration"]), dims="obs_id"
            )
            mean_durations = pymc.MutableData(
                "mean_durations", pymc.floatX(empty_df["duration"]), dims="obs_id"
            )
            pains = pymc.MutableData(
                "pains", pymc.floatX(empty_df["duration"]), dims="obs_id"
            )
            mu += type_intercept[types]

            intensity_coefficients = pymc.Normal(
                "intensity_coefficients", mu=0, sigma=1, shape=2
            )
            duration_coefficients = pymc.Normal(
                "duration_coefficients", mu=0, sigma=1, shape=2
            )
            pain_coefficients = pymc.Normal("pain_coefficients", mu=0, sigma=1, shape=2)
            mu += (
                intensity_coefficients[0] + intensity_coefficients[1] * mean_intensities
            ) * intensities
            mu += (
                duration_coefficients[0] + duration_coefficients[1] * mean_durations
            ) * durations
            mu += (
                (pain_coefficients[0] + pain_coefficients[1] * pains)
                * intensities
                * durations
            )

            observed_outcomes = pymc.MutableData(
                "observed_outcomes", empty_df["pain_reduction"], dims="obs_id"
            )
            outcome = pymc.Normal(
                "outcome",
                mu=mu,
                sigma=pymc.Exponential(name="sigma", lam=1),
                observed=observed_outcomes,
                dims="obs_id",
            )

    def drop_non_compliant_rows(self, df):
        compliant_true_df = df[df['compliant'] == True].copy()
        return compliant_true_df
        
    def update_posterior(self, history, _):
        df = self.drop_non_compliant_rows(history.to_df())

        #print(df)
        if not self.model:
            self.setup_model()

        with self.model:
            pymc.set_data(
                {
                    "types": df["type"],
                    "durations": df["duration"],
                    "mean_durations": df["mean_duration"],
                    "intensities": df["intensity"],
                    "mean_intensities": df["mean_intensity"],
                    "pains": df["current_pain"],
                    "observed_outcomes": df["pain_reduction"],
                }
            )
            self.trace = pymc.sample(2000, progressbar=False)

    def approximate_max_probabilities(self, number_of_treatments, context):
        assert (
            self.trace is not None
        ), "You called `approximate_max_probabilites` without updating the posterior"

        df = pandas.DataFrame(
            [context] * number_of_treatments,
        )
        df["activity_index"] = range(number_of_treatments)
        activity_df = pandas.DataFrame(self.possible_actions)

        df = pandas.concat([df, pandas.DataFrame(self.possible_actions)], axis=1)

        # Eliminate duplicate columns
        df = df.loc[:, ~df.columns.duplicated()].copy()

        with self.model:
            pymc.set_data(
                {
                    "types": df["type"],
                    "durations": df["duration"],
                    "mean_durations": df["mean_duration"],
                    "intensities": df["intensity"],
                    "mean_intensities": df["mean_intensity"],
                    "pains": df["current_pain"],
                }
            )
            pymc.sample_posterior_predictive(
                self.trace,
                var_names=["outcome"],
                extend_inferencedata=True,
            )

        max_indices = arviz.extract(self.trace.posterior_predictive).outcome.argmax(
            dim="obs_id"
        )
        bin_counts = numpy.bincount(max_indices, minlength=number_of_treatments)
        return bin_counts / numpy.sum(bin_counts)

## Agent specification

In [ ]:
# Wrapped in ComposedPolicy to ensure same behavior as other options
fixed_policy = ComposedPolicy(
    policies=[
        SelectionPolicy(
            policy=FixedPolicy(number_of_actions=len(possible_actions)),
            columns=columns,
            possible_actions=possible_actions,
        ),
        SelectionPolicy(
            policy=FixedPolicy(number_of_actions=len(possible_actions)),
            columns=columns,
            possible_actions=possible_actions,
        ),
        SelectionPolicy(
            policy=FixedPolicy(number_of_actions=len(possible_actions)),
            columns=columns,
            possible_actions=possible_actions,
        ),
    ],
    durations=durations,
    number_of_actions=len(possible_actions),
    outcome_name="pain_reduction",
)

In [ ]:
physical_exercise_policy_A = ComposedPolicy(
    policies=[
        SelectionPolicy(
            policy=FixedPolicy(number_of_actions=len(possible_actions)),
            columns=columns,
            possible_actions=possible_actions,
        ),
        SelectionPolicy(
            policy=FixedPolicy(number_of_actions=len(possible_actions)),
            columns=columns,
            possible_actions=possible_actions,
        ),
        SelectionPolicy(
            policy=ThompsonSampling(
                inference_model=PhysicalExerciseModel(
                    dimension_for_type=4, possible_actions=possible_actions
                ),
                number_of_actions=len(possible_actions),
                posterior_update_interval=posterior_update_interval,
                outcome_name="pain_reduction",
            ),
            columns=columns,
            possible_actions=possible_actions,
        ),
    ],
    durations=durations,
    number_of_actions=len(possible_actions),
    outcome_name="pain_reduction",
)

In [ ]:
physical_exercise_policy_B = ComposedPolicy(
    policies=[
        SelectionPolicy(
            policy=FixedPolicy(number_of_actions=len(possible_actions)),
            columns=columns,
            possible_actions=possible_actions,
        ),
        SelectionPolicy(
            policy=ThompsonSampling(
                inference_model=PhysicalExerciseModel(
                    dimension_for_type=4, possible_actions=possible_actions
                ),
                number_of_actions=len(possible_actions),
                posterior_update_interval=posterior_update_interval,
                outcome_name="pain_reduction",
            ),
            columns=columns,
            possible_actions=possible_actions,
        ),
        SelectionPolicy(
            policy=FixedPolicy(number_of_actions=len(possible_actions)),
            columns=columns,
            possible_actions=possible_actions,
        ),
    ],
    durations=durations,
    number_of_actions=len(possible_actions),
    outcome_name="pain_reduction",
)

## Simulation Run

In [ ]:
# This is time intensive!

all_scenarios = [scenario_I, scenario_II, scenario_III, scenario_IV, scenario_V, scenario_VI, scenario_VII]
scenarios = all_scenarios

# If run in parallel, only calculate one scenario
if parallel_scenario != -1:
    scenarios = [all_scenarios[parallel_scenario]]

configuration_specification = {
    "policy": [fixed_policy, physical_exercise_policy_A, physical_exercise_policy_B],
    #"policy": [fixed_policy],
    "model_from_patient_id": scenarios,
    #"model_from_patient_id": [scenario_VII],
}


def generate_configuration_cross_product(configuration_specification):
    configuration_dimensions = [
        list(range(len(item))) for item in configuration_specification.values()
    ]
    configuration_indices = list(itertools.product(*configuration_dimensions))
    parameter_names = list(configuration_specification.keys())
    parameter_value_array = list(configuration_specification.values())
    configurations = [
        {
            parameter_names[index]: parameter_value_array[index][value]
            for index, value in enumerate(configuration_index)
        }
        for configuration_index in configuration_indices
    ]
    return configurations


configurations = generate_configuration_cross_product(configuration_specification)

calculated_series = []
for configuration in tqdm(configurations):
    print(configuration)
    result = SeriesOfSimulationsRunner(n_patients=n_patients, **configuration).simulate(
        length
    )
    calculated_series.append({"configuration": result.configuration, "result": result})

config_to_simulation_data = {
    str(simulation.configuration): simulation
    for d in calculated_series
    for simulation in d["result"].simulations
}

### Write to disk

In [ ]:
file_appendix = ""
if parallel_scenario != -1:
    file_appendix = "_parallel_scenario_" + str(parallel_scenario)

filename = f"2023-11-08-data{file_appendix}.json"
with open(filename, "w") as file:
    file.write(jsonpickle.encode([calculated_series, config_to_simulation_data]))

In [ ]:
print(datetime.datetime.now()) # Helps to see how long jobs took and when they were started